#P0. Introducción -PLN

---

En la creación de redes neuronales necesitamos dos tipos de IA, para reconocer patrones o generar nuevos:

*   Las que no tienen memoria, identifica el patrón y ya!...ejemplo las de visión artificial
*   Las de memoria corta (Long Short Term Memory)...PLN
*   Las que requieren mucha memoria (aprenden casi todo...BERT)...PLN y visión artificial.



**Caso de estudio: generación de texto**

---


Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos 

Para este ejemplo usaremos “*Character level language model*” propuesto por Andrej Karpathy en su artículo "*The Unreasonable Effectiveness of Recurrent Neural Networks*"(y parcialmente basado en su implementado en el tutorial "*Generate text with an RNN*" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.


---
https://unipython.com/generacion-de-textos-con-inteligencia-artificial/


#Ejemplo 2: generar texto de cuentos, usando Keras

##P0. importar librerias

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import sys

##P0. Descarga y preprocesado de los datos

In [ ]:
fileDL= tf.keras.utils.get_file('LaInterpretacionDeLosSueñosSigmundFreud.txt','https://raw.githubusercontent.com/Moisesdv10/Deep-Learning/main/Segundo%20corte/PLN/Datasets/Panel_Txt_Files/LaInterpretacionDeLosSue%C3%B1osSigmundFreud.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')
texto = texto.lower()

1310720/1300981 [==============================] - 0s 0us/step


##P1. entendiendo el texto

In [ ]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:1293168 caracteres
el texto esta compuesto de estos :80 caracteres
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', '+', ',', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '>', '?', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', 'ª', '«', 'º', '»', '¿', 'ß', 'à', 'â', 'ä', 'æ', 'ç', 'è', 'ê', 'î', 'ñ', 'ô', 'ö', 'û', 'ü', '…', '�']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'la interpretacion de los sueños         \n    sigmund freud \nsigmund freud \nla interpretacion de los '
array([43, 32,  1, 40, 45, 51, 36, 49, 47, 49, 36, 51, 32, 34, 40, 46, 45,
        1, 35, 36,  1, 43, 46, 50,  1, 50, 52, 36, 73, 46, 50,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  0,  1,  1,  1,  1, 50, 40, 38, 44, 52, 45,
       35,  1, 37, 49, 36, 52, 35,  1,  0, 50, 40, 38, 44, 52, 45, 35,  1,
       37, 49, 36, 52, 35,  1,  0, 43, 32,  1, 40, 45, 51, 36, 49, 47, 49,
       36, 51, 32, 34, 40, 46, 45,  1, 35, 36,  1, 43, 46, 50,  1])


##P3. preparar los datos para ser usados en la RNN

In [ ]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'la interpretacion de los sueños         \n    sigmund freud \nsigmund freud \nla interpretacion de los sueños \n  \n18989 [1900] \nla interpretacion de los s'
'ueños         \n    sigmund freud \nprefacio a la primera edicion \n(1900) \nal proponerme exponer la interpretacion de los sueños no creo haber trascendid'
'o los \nambitos del interes neuropatologico, pues, el examen psicologico nos presenta el sueño co\nmo primer eslabon de una serie de fenomenos psiquicos '
'anormales, entre cuyos elementos \nsubsiguientes, las fobias histericas y las formaciones obsesivas y delirantes, conciernen al \nmedico por motivos prac'
'ticos. desde luego, como ya lo demostraremos, el sueño no puede \npretender analoga importancia practica; pero tanto mayor es su valor teorico como para'
'dig\nma, al punto que quien no logre explicarse la genesis de las imagenes oniricas, se esforzara \nen vano por comprender las fobias, las ideas obsesiva'
's, los delirios, y por ejercer sobre esa \nestos fenomenos

###P3.1 separar los datos en agrupamientos (batches)

In [ ]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'la interpretacion de los sueños         \n    sigmund freud \nsigmund freud \nla interpretacion de los sueños \n  \n18989 [1900] \nla interpretacion de los '
Target data:  'a interpretacion de los sueños         \n    sigmund freud \nsigmund freud \nla interpretacion de los sueños \n  \n18989 [1900] \nla interpretacion de los s'


In [ ]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=build_model(vocab_size=vocab_size,
                  embedding_dim=embedding_dim,
                  rnn_units=rnn_units,
                  batch_size=BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=
#    -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           20480     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 80)            82000     
                                                                 
Total params: 5,349,456
Trainable params: 5,349,456
Non-trainable params: 0
_________________________________________________________________


##P4. Entrenando la RNN

In [ ]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
#con los argumentos por defecto del optimizador Adam.  
model.compile(optimizer='adam',loss=loss)

###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.


In [ ]:
checkpoint_dir='/content/checkpointV2'
checkpoint_prefix= os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

###P4.2 entrenando

In [11]:
EPOCHS=500
history=model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/500
133/133 [==============================] - 39s 231ms/step - loss: 2.4922
Epoch 2/500
133/133 [==============================] - 33s 234ms/step - loss: 1.9017
Epoch 3/500
133/133 [==============================] - 33s 236ms/step - loss: 1.5886
Epoch 4/500
133/133 [==============================] - 34s 239ms/step - loss: 1.4000
Epoch 5/500
133/133 [==============================] - 34s 239ms/step - loss: 1.2928
Epoch 6/500
133/133 [==============================] - 34s 239ms/step - loss: 1.2252
Epoch 7/500
133/133 [==============================] - 34s 240ms/step - loss: 1.1777
Epoch 8/500
133/133 [==============================] - 34s 241ms/step - loss: 1.1404
Epoch 9/500
133/133 [==============================] - 34s 239ms/step - loss: 1.1061
Epoch 10/500
133/133 [==============================] - 34s 241ms/step - loss: 1.0749
Epoch 11/500
133/133 [==============================] - 34s 240ms/step - loss: 1.0444
Epoch 12/500
133/133 [==============================] - 34s 241

##P5. Generando texto nuevo usando la RNN

In [12]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [13]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.5  
  #entre más alta la temperatura más creatividad al modelo, pero tambien
  #más errores ortograficos.
  model.reset_states()
  #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

###P5.1 generando texto 

In [14]:
print(generate_text(model, start_string=u"la interpretacion de los sueños"))

la interpretacion de los sueños         
    sigmund freud 
el simbolismo prepara extrañar la relacion de este sueño. 
4) la representacion del material reciente, se halla en la mayor 
inversion a su montancia). 
la explicacion del sueño se agregan como una de las fantasias en absoluto de la inversion y se 
hallan mostradas a pesar de la general libidos de alguien que el material distinto ha laguna de al
minicamente de por que el sueño se refiere, excitando a una persona con una señala». 
la elaboracion de los sueños pueden m


##P6.exportando modelo

In [15]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentosV2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modelRNN_cuentosV2_pesos.hdf5")
model.save('modelRNN_cuentosV2.h5')
print("modelo salvado en disco")

modelo salvado en disco
